In [1]:
import numpy as np

#--- Parametreler ---
N = 100          # Düğüm sayısı
p = 0.1          # Erdős–Rényi bağlantı olasılığı
theta = 2.0
beta = 2.0
max_iter = 100
tol = 1e-5

# Rastgele alpha_i ve gelir e_i
alpha = np.random.uniform(0, 1, N)
e = np.random.uniform(100, 200, N)

#--- Ağ oluşturma (Erdős–Rényi) ---
adj_matrix = (np.random.rand(N, N) < p).astype(int)
# Undirected ve diagonal=0
for i in range(N):
    adj_matrix[i,i] = 0
for i in range(N):
    for j in range(i+1, N):
        adj_matrix[j,i] = adj_matrix[i,j]

neighbors = []
for i in range(N):
    Ni = [j for j in range(N) if adj_matrix[i,j] == 1]
    neighbors.append(Ni)

#--- Yardımcı fonksiyon: dU_i/dc_i ---
def dUi_dc(i, c_i, c_other):
    """
    i oyuncusunun faydasının c_i'ye göre türevi,
    c_other dizisinde diğer c_j'ler sabit.
    """
    Ni = neighbors[i]
    # Kendi tüketim kısmı
    # e_i - c_i^theta + (beta/N_i) sum_{j in Ni} c_j
    Ni_size = len(Ni)
    sum_neigh = sum(c_other[j] for j in Ni)
    denom_i = e[i] - (c_i**theta) + (beta/Ni_size)*sum_neigh

    # Türev: -theta * c_i^(theta-1) / denom_i
    if denom_i <= 0:
        # Argüman negatife düşerse log tanımsız olur.
        # Bu, "çok büyük c_i" anlamına gelebilir.
        return -1e10  # "çok negatif"

    part_self = -(theta * c_i**(theta-1)) / denom_i

    # Komşuların tüketiminden gelen kısım
    part_others = 0.0
    for j in Ni:
        # e_j - c_other[j]^theta + (beta/|Nj|) * sum_{k in Nj} c_other[k]
        Nj = neighbors[j]
        Nj_size = len(Nj)
        sum_Nj = sum(c_other[k] for k in Nj)
        denom_j = e[j] - (c_other[j]**theta) + (beta/Nj_size)*sum_Nj

        if denom_j > 0:
            # Türev: alpha_i * (beta/Nj_size) / denom_j
            # i in Nj varsayımı (undirected)
            part_others += alpha[i] * (beta/Nj_size) / denom_j
        else:
            # Argüman sıfır veya negatif
            part_others += alpha[i] * 1e10  # cezalandırıcı +∞

    return part_self + part_others

# Numerik yöntem: Bisection veya Newton ile dUi/dc_i = 0 noktasını bul
def best_response(i, c_other, left=0.0, right=1000.0):
    """
    i oyuncusunun en iyi tepkisini numerik olarak bul (bisection),
    dUi/dc_i=0 kökünü arıyoruz.
    """
    for _ in range(50):  # 50 yineleme
        mid = 0.5*(left + right)
        f_mid = dUi_dc(i, mid, c_other)

        if f_mid > 0:
            # f_mid>0 => optimum daha büyük c_i'da olabilir
            left = mid
        else:
            # f_mid<0 => optimum daha küçük c_i'da
            right = mid

    return 0.5*(left + right)

#--- BR iterasyonu ---
c = np.zeros(N)  # başlangıç

for t in range(max_iter):
    c_new = np.zeros(N)
    # Her oyuncu, digerlerinin c_j(t) degerine karsilik en iyi tepkiyi bulur
    for i in range(N):
        # c_other = c_new'yu değil c'yi kullanıyoruz (EŞZAMANLI GÜNCELLEME)
        c_new[i] = best_response(i, c)

    diff = np.linalg.norm(c_new - c)
    if diff < tol:
        print(f"Converged at iteration {t}, diff={diff:.6e}")
        break
    c = c_new

print("Nash equilibrium candidate c*:", c)


Converged at iteration 3, diff=1.451468e-06
Nash equilibrium candidate c*: [0.3362198  0.6441284  0.22276182 0.66493179 0.3908019  0.07023473
 0.70620744 0.73768297 0.83317061 0.39854991 0.625593   0.12115814
 0.17440289 0.67854606 0.83959417 1.32382904 0.99203278 0.25874169
 0.76043167 0.89222288 0.03866015 0.15863886 0.40910566 0.46140175
 0.20186751 0.18066697 0.11369171 0.19261615 0.39113678 0.97596891
 0.43152248 0.17303421 0.22030069 0.42648302 0.50755695 0.34641327
 0.75672111 0.15112755 0.49813171 0.45029831 0.15485754 0.11928269
 0.5786921  0.36918722 0.21486848 0.34715692 0.55809003 0.18685198
 0.93234304 0.58277606 0.29766928 0.19360683 0.22671702 0.89327626
 0.23500781 0.12785258 0.26100121 0.09526274 1.31534064 0.10004641
 1.01986703 0.30005741 0.70633454 0.03255667 0.70771585 0.42829394
 1.45603701 0.24478752 0.53403573 0.04005247 0.82836666 0.32285168
 1.02933231 0.81163876 0.75544427 0.41050445 0.84013478 0.2069341
 0.95206966 1.73509385 0.27656382 0.9718954  0.3566188 